In [1]:
from selenium import webdriver
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd 
from datetime import datetime, timedelta 
from datetime import date
import urllib.robotparser 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [2]:
# YOUR CODE HERE
def denied(response):
    if "\"incomplete_results\":true" in response.text:
        return False
    else:
        return True
requests_cache.install_cache(expire_after = None, filter_fn = denied)

In [3]:
url = "https://soundcloud.com/search/people?q=*"
service = Service("chromedriver.exe")
#cái này setup theo mấy trang nó chỉ!!
driver1 = webdriver.Chrome(service = service)
driver1.get(url)
time.sleep(1)
soup = BeautifulSoup(driver1.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver1.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

driver2 = webdriver.Chrome(service = service)
driver2.get(url)
time.sleep(1)
soup = BeautifulSoup(driver2.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver2.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

In [4]:
def write_file(page_source):    
    soup = BeautifulSoup(page_source)
    sound_item = soup.find_all("li",{"class","soundList__item"})

    count_item = 0
    for item in sound_item:
        if count_item == 7:
            break
        count_item += 1
        item_url = item.find("a",{"class","sound__coverArt"})["href"]
#             print(item_url)
        sound_url = "https://soundcloud.com/" + item_url
        req = requests.get(sound_url)
        soup = BeautifulSoup(req.text, "lxml")
        search = soup.find_all("script",{"":""})
        for script in search:
            if "window.__sc_hydration" in script.text:
                search = script
                break

        res = json.loads(search.text[24:-1])
        hydratable_sound = res[6]["data"]
#             print(hydratable_sound)
        for i in hydratable_sound.keys():
            if hydratable_sound[i] == "":
                hydratable_sound[i] = "None"

        for attr in attributes:
            output.write(str(hydratable_sound[attr]).replace(",",";"))
            output.write(",")
        output.write("\n")

In [5]:
users_urls = []
users_urls_file = "good_user.txt"
with open(users_urls_file) as f:
    for line in f:
        users_urls.append(line.replace("\n",""))
        
attributes = ["artwork_url","commentable","comment_count","created_at","downloadable"
             ,"download_count","duration","full_duration","embeddable_by","genre"
             ,"has_downloads_left","id","kind","label_name","last_modified","license"
             ,"likes_count","permalink_url","playback_count","public","purchase_title"
             ,"purchase_url","release_date","reposts_count","sharing","state","streamable"
             ,"tag_list","title","track_format","user_id","visuals","waveform_url","display_date"
             ,"track_authorization","monetization_model","policy"]
output = open("track.csv","w",encoding = "utf-8")

for attr in attributes:
    output.write(attr)
    output.write(",")
output.write("\n")
step = 0
users_urls.append("")

driver1.get(users_urls[0]+"/tracks")
time.sleep(3)
for url in users_urls:
    try:
        step+=1
        if step % 100==0:
            print("%")
        else:
            if step %10==0:
                print("#",end="")
#                 break

        if step == 1: 
            continue
        
        if url=="":
            if i%2==0:
                write_file(driver1.page_source)
            else:
                write_file(driver2.page_source)
        
        if step%2==0:
            driver2.get(url+"/tracks")
            write_file(driver1.page_source)
        else:
            driver1.get(url+"/tracks")
            write_file(driver2.page_source)
#         time.sleep(1.5)
        
        

    except:
        continue
    
print("finish")
output.close()

#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#########%
#######finish
